#Μηχανική Μάθηση σε Πολυμεσικά Δεδομένα - Τελική Εργασία
##Μάριος Σαρρηγιάννης (mtn2116)

Στα πλαίσια του μαθήματος, αναφερθήκαμε στο κομμάτι του Speech Recognition. Σε αυτή την εργασία, γίνεται μια προσπάθεια ανάπτυξης ενός μοντέλου βασισμένο στην αρχιτεκτονική του [Deep Speech 2](https://arxiv.org/pdf/1512.02595v1.pdf), το οποίο χρησιμοποιεί τα melgrams του ήχου, τα οποία τα επεξεργάζεται ένα δίκτυο από N ResCNN layers και παράγει τα features τα οποία οδηγούνται σε N BiRNN layers τα οποία κάνουν predict τους χαρακτήρες της φράσης που ειπώθηκε. Οι χαρακτήρες οδηγούνται σε έναν decoder ο οποίος παράγει την τελική φράση. Το dataset είναι το [LibriSpeech ASR](https://www.openslr.org/12) corpus, το οποίο είναι σύνηθες corpus για τη σύγκριση ASR μοντέλων. Το μοντέλο αξιολογείται ως προς το [word error rate(wer)](https://huggingface.co/spaces/evaluate-metric/wer) και το [character error rate(cer)](https://huggingface.co/spaces/evaluate-metric/cer). Η υλοποίηση βασίζεται στο παρακάτω [tutorial](https://www.assemblyai.com/blog/end-to-end-speech-recognition-pytorch/).

####Αρχικά, φορτώνονται οι βιβλιοθήκες που θα χρησιμοποιηθούν

In [ ]:
!pip install evaluate jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |████████████████████████████████| 86 kB 6.1 MB/s 
     |████████████████████████████████| 362 kB 45.7 MB/s 
     |████████████████████████████████| 140 kB 64.1 MB/s 
     |████████████████████████████████| 212 kB 68.6 MB/s 
     |████████████████████████████████| 1.1 MB 60.0 MB/s 
     |████████████████████████████████| 596 kB 43.1 MB/s 
     |████████████████████████████████| 127 kB 69.4 MB/s 
     |████████████████████████████████| 50 kB 9.5 MB/s 
     |████████████████████████████████| 144 kB 68.9 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
     |████████████████████████████████| 271 kB 53.2 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149877 sha256=16f848d13739b0c7fd43a35934b74a70df64fc613b563c4c6cdb8ade61b51094
  Stored in directory: /ro

In [ ]:
import os
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
import torch.nn.functional as F
import torchaudio
import numpy as np
from evaluate import load

####Στη συνέχεια, μεταφορτώνεται το dataset

In [ ]:
if not os.path.isdir("./data"):
        os.makedirs("./data")

train_dataset = torchaudio.datasets.LIBRISPEECH("./data", url="train-clean-100", download=True)
test_dataset = torchaudio.datasets.LIBRISPEECH("./data", url="test-clean", download=True)

  0%|          | 0.00/5.95G [00:00<?, ?B/s]

  0%|          | 0.00/331M [00:00<?, ?B/s]

#### Μετα, ορίζεται η κλάση που κωδικοποιεί τους χαρακτήρες σε ακεραίους, και παρέχει 2 βοηθητικές μεθόδους για το conversion από σειρά ακεραίων σε κείμενο και αντίστροφα

In [ ]:
class TextTransform:
    def __init__(self):
        char_map_str = """
        ' 0
        <SPACE> 1
        a 2
        b 3
        c 4
        d 5
        e 6
        f 7
        g 8
        h 9
        i 10
        j 11
        k 12
        l 13
        m 14
        n 15
        o 16
        p 17
        q 18
        r 19
        s 20
        t 21
        u 22
        v 23
        w 24
        x 25
        y 26
        z 27
        """
        self.char_map = {}
        self.index_map = {}
        for line in char_map_str.strip().split('\n'):
            ch, index = line.split()
            self.char_map[ch] = int(index)
            self.index_map[int(index)] = ch
        self.index_map[1] = ' '

    def text_to_int(self, text):
        int_sequence = []
        for c in text:
            if c == ' ':
                ch = self.char_map['<SPACE>']
            else:
                ch = self.char_map[c]
            int_sequence.append(ch)
        return int_sequence

    def int_to_text(self, labels):
        string = []
        for i in labels:
            string.append(self.index_map[i])
        return ''.join(string).replace('<SPACE>', ' ')


text_transform = TextTransform()

####Φορτώνουμε τις μετρικές αξιολόγησης από το library τους

Το wer και το cer είναι χρήσιμες μετρικές αξιολόγισης ενώς ASR συστήματος. Το πρώτο μετράει τις διαφορές ως προς τις λέξεις και το δεύτερο ως προς του χαρακτήρες. Οι μετρικές ορίζονται ως $(S + D + I) / N = (S + D + I) / (S + D + C)$ όπου S είναι ο αριθμός αλλαγών θέσεων , D είναι ο αριθμός διαγραφών, I είναι ο αριθμός εισαγωγών, C είναι ο αριθμός σωστών λέξεων και N είναι ο συνολικός αριθμός των λέξεων του ground truth (N=S+D+C) (το wer ως προς τις λέξεις ενώ το cer ως προς τους χαρακτήρες).

In [ ]:
wer = load("wer")
cer = load("cer")

#### Preprocessing - Σε αυτό το βήμα, ορίζουμε τα preprocessing steps για τα train και τα validation δεδομένα. Στα validation δεδομένα, απλά εξάγονται τα melgrams. Στα training δεδομένα, εφαρμόζεται η τεχνική [SpecAugment](https://arxiv.org/abs/1904.08779) για data augmentation κατα την οποία, αφαιρούνται από το spectogram ορισμένες (ομοιόμορφα κατανεμημένες) συχνότητες και ορισμένα (ομοιόμορφα κατανεμημένα) χρονικά διαστήματα.

In [ ]:
train_audio_transforms = nn.Sequential(
    torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=128),
    torchaudio.transforms.FrequencyMasking(freq_mask_param=30),
    torchaudio.transforms.TimeMasking(time_mask_param=100)
)

valid_audio_transforms = torchaudio.transforms.MelSpectrogram()

def data_processing(data, data_type="train"):
    spectrograms = []
    labels = []
    input_lengths = []
    label_lengths = []
    for (waveform, _, utterance, _, _, _) in data:
        if data_type == 'train':
            spec = train_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        elif data_type == 'valid':
            spec = valid_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        else:
            raise Exception('data_type should be train or valid')
        spectrograms.append(spec)
        label = torch.Tensor(text_transform.text_to_int(utterance.lower()))
        labels.append(label)
        input_lengths.append(spec.shape[0]//2)
        label_lengths.append(len(label))

    spectrograms = nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).unsqueeze(1).transpose(2, 3)
    labels = nn.utils.rnn.pad_sequence(labels, batch_first=True)

    return spectrograms, labels, input_lengths, label_lengths

use_cuda = torch.cuda.is_available()
torch.manual_seed(7)
device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = data.DataLoader(dataset=train_dataset,
                            batch_size=20,
                            shuffle=True,
                            collate_fn=lambda x: data_processing(x, 'train'),
                            **kwargs)
test_loader = data.DataLoader(dataset=test_dataset,
                            batch_size=20,
                            shuffle=False,
                            collate_fn=lambda x: data_processing(x, 'valid'),
                            **kwargs)

/usr/local/lib/python3.7/dist-packages/torchaudio/functional/functional.py:508: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  "At least one mel filterbank has all zero values. "


#### ResCNN - Το [ResidualCNN](https://arxiv.org/abs/1512.03385) layer με layer normalization το οποίο εξάγει τα χαρακτηριστικά απο τα MelGrams

In [ ]:
class CNNLayerNorm(nn.Module):
    def __init__(self, n_feats):
        super(CNNLayerNorm, self).__init__()
        self.layer_norm = nn.LayerNorm(n_feats)

    def forward(self, x):
        # x (batch, channel, feature, time)
        x = x.transpose(2, 3).contiguous() # (batch, channel, time, feature)
        x = self.layer_norm(x)
        return x.transpose(2, 3).contiguous() # (batch, channel, feature, time) 


class ResidualCNN(nn.Module):
    def __init__(self, in_channels, out_channels, kernel, stride, dropout, n_feats):
        super(ResidualCNN, self).__init__()

        self.cnn1 = nn.Conv2d(in_channels, out_channels, kernel, stride, padding=kernel//2)
        self.cnn2 = nn.Conv2d(out_channels, out_channels, kernel, stride, padding=kernel//2)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.layer_norm1 = CNNLayerNorm(n_feats)
        self.layer_norm2 = CNNLayerNorm(n_feats)

    def forward(self, x):
        residual = x  # (batch, channel, feature, time)
        x = self.layer_norm1(x)
        x = F.gelu(x)
        x = self.dropout1(x)
        x = self.cnn1(x)
        x = self.layer_norm2(x)
        x = F.gelu(x)
        x = self.dropout2(x)
        x = self.cnn2(x)
        x += residual
        return x # (batch, channel, feature, time)

#### Το BiRNN layer υλοποιημένο με GRUs το οποίο λαμβάνει τα features του ResCNN για να προβλέψει τους χαρακτήρες

In [ ]:
class BidirectionalGRU(nn.Module):

    def __init__(self, rnn_dim, hidden_size, dropout, batch_first):
        super(BidirectionalGRU, self).__init__()

        self.BiGRU = nn.GRU(
            input_size=rnn_dim, hidden_size=hidden_size,
            num_layers=1, batch_first=batch_first, bidirectional=True)
        self.layer_norm = nn.LayerNorm(rnn_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.layer_norm(x)
        x = F.gelu(x)
        x, _ = self.BiGRU(x)
        x = self.dropout(x)
        return x

#### Ορίζουμε το μοντέλο που χρησιμοποιεί τα παραπάνω layers στην παρακάτω διάταξη: 

####N ResCNN layers -> Linear Layer -> N BiRNN layers -> N linear layers -> Softmax Output

#### Το output του δικτύου είναι μια κατανομή πιθανοτήτων χαρακτήρων που υπόθηκαν την κάθε χρονική στιγμή

In [ ]:
class SpeechRecognitionModel(nn.Module):
    
    def __init__(self, n_cnn_layers, n_rnn_layers, rnn_dim, n_class, n_feats, stride=2, dropout=0.1):
        super(SpeechRecognitionModel, self).__init__()
        n_feats = n_feats//2
        self.cnn = nn.Conv2d(1, 32, 3, stride=stride, padding=3//2)  # cnn for extracting heirachal features

        # n residual cnn layers with filter size of 32
        self.rescnn_layers = nn.Sequential(*[
            ResidualCNN(32, 32, kernel=3, stride=1, dropout=dropout, n_feats=n_feats) 
            for _ in range(n_cnn_layers)
        ])
        self.fully_connected = nn.Linear(n_feats*32, rnn_dim)
        self.birnn_layers = nn.Sequential(*[
            BidirectionalGRU(rnn_dim=rnn_dim if i==0 else rnn_dim*2,
                             hidden_size=rnn_dim, dropout=dropout, batch_first=i==0)
            for i in range(n_rnn_layers)
        ])
        self.classifier = nn.Sequential(
            nn.Linear(rnn_dim*2, rnn_dim),  # birnn returns rnn_dim*2
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(rnn_dim, n_class)
        )

    def forward(self, x):
        x = self.cnn(x)
        x = self.rescnn_layers(x)
        sizes = x.size()
        x = x.view(sizes[0], sizes[1] * sizes[2], sizes[3])  # (batch, feature, time)
        x = x.transpose(1, 2) # (batch, time, feature)
        x = self.fully_connected(x)
        x = self.birnn_layers(x)
        x = self.classifier(x)
        return x

model = SpeechRecognitionModel(n_cnn_layers=1, n_rnn_layers=1, rnn_dim=512, n_class=29, n_feats=128, stride=2, dropout=0.1).to(device)

print(model)
print('Num Model Parameters', sum([param.nelement() for param in model.parameters()]))

SpeechRecognitionModel(
  (cnn): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (rescnn_layers): Sequential(
    (0): ResidualCNN(
      (cnn1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (cnn2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (layer_norm1): CNNLayerNorm(
        (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
      (layer_norm2): CNNLayerNorm(
        (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
    )
  )
  (fully_connected): Linear(in_features=2048, out_features=512, bias=True)
  (birnn_layers): Sequential(
    (0): BidirectionalGRU(
      (BiGRU): GRU(512, 512, batch_first=True, bidirectional=True)
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (classifier): S

####Ορίζουμε τον greedy decoder ο οποίος διαλέγει τον χαρακτήρα με την μεγαλύτερη πιθανότητα την κάθε χρονική στιγμή.

In [ ]:
def GreedyDecoder(output, labels, label_lengths, blank_label=28, collapse_repeated=True):
	arg_maxes = torch.argmax(output, dim=2)
	decodes = []
	targets = []
	for i, args in enumerate(arg_maxes):
		decode = []
		targets.append(text_transform.int_to_text(labels[i][:label_lengths[i]].tolist()))
		for j, index in enumerate(args):
			if index != blank_label:
				if collapse_repeated and j != 0 and index == args[j -1]:
					continue
				decode.append(index.item())
		decodes.append(text_transform.int_to_text(decode))
	return decodes, targets

####Τέλος, ορίζουμε συναρτήσεις για train και test.

In [ ]:
def train(model, device, train_loader, criterion, optimizer, scheduler, epoch):
    model.train()
    data_len = len(train_loader.dataset)
    for batch_idx, _data in enumerate(train_loader):
        spectrograms, labels, input_lengths, label_lengths = _data 
        spectrograms, labels = spectrograms.to(device), labels.to(device)

        optimizer.zero_grad()

        output = model(spectrograms)  # (batch, time, n_class)
        output = F.log_softmax(output, dim=2)
        output = output.transpose(0, 1) # (time, batch, n_class)

        loss = criterion(output, labels, input_lengths, label_lengths)
        loss.backward()

        optimizer.step()
        scheduler.step()
        if batch_idx % 100 == 0 or batch_idx == data_len:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(spectrograms), data_len,
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader, criterion):
    print('\nevaluating...')
    model.eval()
    test_loss = 0
    test_cer, test_wer = [], []
    with torch.no_grad():
        for i, _data in enumerate(test_loader):
            spectrograms, labels, input_lengths, label_lengths = _data 
            spectrograms, labels = spectrograms.to(device), labels.to(device)

            output = model(spectrograms)  # (batch, time, n_class)
            output = F.log_softmax(output, dim=2)
            output = output.transpose(0, 1) # (time, batch, n_class)

            loss = criterion(output, labels, input_lengths, label_lengths)
            test_loss += loss.item() / len(test_loader)

            decoded_preds, decoded_targets = GreedyDecoder(output.transpose(0, 1), labels, label_lengths)
            test_wer.append(wer.compute(predictions=decoded_preds, references=decoded_targets ))
            test_cer.append(cer.compute(predictions=decoded_preds, references=decoded_targets ))
                
    avg_cer = sum(test_cer)/len(test_cer)
    avg_wer = sum(test_wer)/len(test_wer)

    print('Test set: Average loss: {:.4f}, Average CER: {:4f} Average WER: {:.4f}\n'.format(test_loss, avg_cer, avg_wer))  

####Εκτελούμε το πείραμα. Σαν optimizer επιλέγουμε τον [AdamW](https://arxiv.org/abs/1711.05101) ο οποίος βελτιώνει τον adam χρησιμοποιώντας weight decay αντί για L2 regularization και επιτρέπει στο μοντέλο να εκπαιδευτεί ταχύτερα. Σαν scheduler χρησιμοποιούμε τον [One Cycle Learning Rate Scheduler](https://arxiv.org/abs/1708.07120) ο οποίος αυξάνει και στη συνέχεια μειώνει το learning rate. Για να αποφύγουμε να κάνουμε align την έξοδο του δικτύου με την είσοδο, χρησιμοποιούμε το [CTC loss function](https://www.researchgate.net/publication/221346365_Connectionist_temporal_classification_Labelling_unsegmented_sequence_data_with_recurrent_neural_'networks), η οποία αναλαμβάνει το temporal alignment.

In [ ]:
epochs = 5

optimizer = optim.AdamW(model.parameters(), 5e-4)
criterion = nn.CTCLoss(blank=28).to(device)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=5e-4, 
                                        steps_per_epoch=int(len(train_loader)),
                                        epochs=epochs,
                                        anneal_strategy='linear')

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, criterion, optimizer, scheduler, epoch)
    test(model, device, test_loader, criterion)

Train Epoch: 1 [0/28539 (0%)]	Loss: 7.081590
Train Epoch: 1 [2000/28539 (7%)]	Loss: 2.922782
Train Epoch: 1 [4000/28539 (14%)]	Loss: 2.855446
Train Epoch: 1 [6000/28539 (21%)]	Loss: 2.873899
Train Epoch: 1 [8000/28539 (28%)]	Loss: 2.866323
Train Epoch: 1 [10000/28539 (35%)]	Loss: 2.821633
Train Epoch: 1 [12000/28539 (42%)]	Loss: 2.711762
Train Epoch: 1 [14000/28539 (49%)]	Loss: 2.418483
Train Epoch: 1 [16000/28539 (56%)]	Loss: 2.291198
Train Epoch: 1 [18000/28539 (63%)]	Loss: 2.068660
Train Epoch: 1 [20000/28539 (70%)]	Loss: 1.953195
Train Epoch: 1 [22000/28539 (77%)]	Loss: 1.961923
Train Epoch: 1 [24000/28539 (84%)]	Loss: 1.962128
Train Epoch: 1 [26000/28539 (91%)]	Loss: 1.830557
Train Epoch: 1 [28000/28539 (98%)]	Loss: 1.759215

evaluating...
Test set: Average loss: 1.5832, Average CER: 0.474155 Average WER: 0.9475

Train Epoch: 2 [0/28539 (0%)]	Loss: 1.841263
Train Epoch: 2 [2000/28539 (7%)]	Loss: 1.658254
Train Epoch: 2 [4000/28539 (14%)]	Loss: 1.595062
Train Epoch: 2 [6000/28539 (

### Συμπεράσματα-επεκτάσεις

* Το μοντέλο παράγει ενθαρρυντικά αποτελέσματα για το μέγεθός του (4.760.733 παράμετροι) και για τις λίγες εποχές εκπαίδευσης
* Για το πρόβλημα σύνθεσης λέξεων, μια πιθανή λύση θα ήταν να χρησιμοποιηθούν labels λέξεων/συλλαβών με τα μειονεκτήματα ότι θα αύξανε δραματικά τις απαιτήσεις του μοντέλου σε μνήμη και θα δημιουργούσε ισχυρή εξάρτηση στην ποιότητα του vocabulary
* Μια ακόμα λύση στο παραπάνω πρόβλημα θα ήταν η χρήση NLP τεχνικών – π.χ. το κείμενο που παράγεται να διορθώνεται με τη βοήθεια ενός προ-εκπαιδευμένου transformer
* To ASR πρόβλημα πλέον προσεγγίζεται καλύτερα με transformers (wav2vec 2.0) οι οποίοι πετυχαίνουν WER~5% ενώ ο άνθρωπος 4%